In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch


In [ ]:
mtransform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize((224,224)),
  transforms.ToTensor()  
])

dataset = datasets.ImageFolder(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Clean Data\Clean Data", transform=transform)
data_loader = torch.utils.data.DataLoader(dataset)

training_stds = []
training_means = []

for image, label in data_loader:
    training_means.append(image.mean(axis=(0, 2, 3)).numpy())
    training_stds.append(image.std(axis=(0, 2, 3)).numpy())
    
training_means = np.array(training_means).mean(axis=0)
training_stds = np.array(training_stds).mean(axis=0)

In [ ]:
my_transform = transforms.Compose([
  transforms.Resize((224,224),interpolation=2),
  transforms.ToTensor(),
  transforms.Normalize([0.6643941, 0.56042176, 0.44903898],[0.31932876, 0.3313484, 0.34955227])
])
dataset = datasets.ImageFolder(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Clean Data\Clean Data", transform=my_transform)

train_data, valid_data, test_data = torch.utils.data.random_split(dataset, [9600,2400,3000], generator=torch.Generator().manual_seed(0))

train_loader = DataLoader(train_data, batch_size=len(train_data),shuffle = True)
valid_loader = DataLoader(valid_data, batch_size=len(valid_data),shuffle = True)
test_loader = DataLoader(test_data, batch_size=len(test_data),shuffle = True)

In [ ]:
torch.save(train_loader, r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\train_loader")
torch.save(valid_loader, r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\valid_loader")
torch.save(test_loader, r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\test_loader")

In [ ]:
train_loader = torch.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\train_loader")
valid_loader = torch.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\valid_loader")
test_loader = torch.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\test_loader")

In [ ]:
def std_mean(image):
    output = []
    #batch size
    #torch.Size([64, 3, 224, 224])
    for j in range(8):
        #arbiturary size
        for k in range(8):
            std = image[:,j*28:(j+1)*28,k*28:(k+1)*28].std()
            avg = image[:,j*28:(j+1)*28,k*28:(k+1)*28].mean()
            output.append(std)
            output.append(avg)
    return output

In [ ]:
def relabel_state(argument):
    switcher = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 1,
        6: 1,
        7: 1,
        8: 1,
        9: 1,
        10:2,
        11:2,
        12:2,
        13:2,
        14:2}
    return switcher[argument.item()]

def relabel_type(argument):
    switcher = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 4,
        5: 0,
        6: 1,
        7: 2,
        8: 3,
        9: 4,
        10:0,
        11:1,
        12:2,
        13:3,
        14:4}
    return switcher[argument.item()]

In [ ]:
fruit_train_images_standard = np.zeros((len(train_data),128))
fruit_train_t_label = np.zeros((len(train_data)))
fruit_train_s_label = np.zeros((len(train_data)))
for image, label in train_loader:
    for i in range(len(label)):
        fruit_train_images_standard[i,:] = std_mean(image[i])
        fruit_train_t_label[i] = relabel_type(label[i])
        fruit_train_s_label[i] = relabel_state(label[i])

np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_train_images_standard.npy", fruit_train_images_standard)
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_train_t_label.npy", fruit_train_t_label)
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_train_s_label.npy", fruit_train_s_label)

In [ ]:
fruit_valid_images_standard = np.zeros((len(valid_data),128))
fruit_valid_t_label = np.zeros((len(valid_data)))
fruit_valid_s_label = np.zeros((len(valid_data)))

for image, label in valid_loader:
    for i in range(len(label)):
        fruit_valid_images_standard[i,:] = std_mean(image[i])
        fruit_valid_t_label[i] = relabel_type(label[i])
        fruit_valid_s_label[i] = relabel_state(label[i])
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_valid_images_standard.npy", fruit_valid_images_standard)
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_valid_t_label.npy", fruit_valid_t_label)
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_valid_s_label.npy", fruit_valid_s_label)

In [ ]:
fruit_test_images_standard = np.zeros((len(test_data),128))
fruit_test_t_label = np.zeros((len(test_data)))
fruit_test_s_label = np.zeros((len(test_data)))
for image, label in test_loader:
    for i in range(len(label)):
        fruit_test_images_standard[i,:] = std_mean(image[i])
        fruit_test_t_label[i] = relabel_type(label[i])
        fruit_test_s_label[i] = relabel_state(label[i])
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_test_images_standard.npy", fruit_test_images_standard)
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_test_t_label.npy", fruit_test_t_label)
np.save(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_test_s_label.npy", fruit_test_s_label)

In [ ]:
fruit_train_images_standard = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_train_images_standard.npy")
fruit_train_t_label = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_train_t_label.npy")
fruit_train_s_label = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_train_s_label.npy")
print(fruit_train_images_standard.shape)

In [ ]:
fruit_valid_images_standard = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_valid_images_standard.npy")
fruit_valid_t_label = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_valid_t_label.npy")
fruit_valid_s_label = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_valid_s_label.npy")
print(fruit_valid_images_standard.shape)

In [ ]:
fruit_test_images_standard = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_test_images_standard.npy")
fruit_test_t_label = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_test_t_label.npy")
fruit_test_s_label = np.load(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Baseline\Numpy\fruit_test_s_label.npy")
print(fruit_test_images_standard.shape)

In [ ]:
if fruit_train_t_images_standard.all() == fruit_train_s_images_standard.all():
    print('x')

x


In [ ]:
rft = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)

rft.fit(fruit_train_images_standard, fruit_train_t_label)

predictions = rft.predict(fruit_valid_images_standard)
accuracy=accuracy_score(predictions ,fruit_valid_t_label)
print('Type Valid Accuracy:', accuracy*100, '%.')

predictions = rft.predict(fruit_test_images_standard)
accuracy=accuracy_score(predictions ,fruit_test_t_label)
print('Type Test Accuracy:', accuracy*100, '%.')

In [ ]:
rfs = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
rfs.fit(fruit_train_images_standard, fruit_train_s_label)
predictions = rfs.predict(fruit_valid_images_standard)
accuracy=accuracy_score(predictions ,fruit_valid_s_label)
print('State Valid Accuracy:', accuracy*100, '%.')

predictions = rfs.predict(fruit_test_images_standard)
accuracy=accuracy_score(predictions ,fruit_test_s_label)
print('State Test Accuracy:', accuracy*100, '%.')

print(predictions)